In [47]:
import pandas as pd
import numpy as np

## 1. u_data, u_item, u_user, genre, occupation -> dataframe으로 변경하기

In [67]:
#u_data-> dataframe으로 변경
u_data = []
with open('./ml-100k/u.data','r',encoding='UTF-8') as f:
    for line in f.readlines():
        u_data.append(line.strip().split('\t'))
    
u_data = pd.DataFrame(u_data, columns=["user id","movie id","rating","timestamp"]) 
#rating을 문자열에서 숫자형으로 바꾸기
u_data['rating'] = pd.to_numeric(u_data['rating'])

In [68]:
#u.item -> dataframe으로 변경
u_item = []
with open("./ml-100k/u.item",'r',encoding='ISO-8859-1') as f:
    for line in f.readlines():
        u_item.append(line.strip().split("|")[:]) #README에서 item의 컬럼은 20개 였으나 해당 데이터에는 24개의 컬럼이 존재하여 4개의 컬럼을 제외시켰습니다
        

movies_info = """movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western"""        

movies_info = movies_info.split(" | ")    

u_item = pd.DataFrame(u_item,columns=movies_info)
#각 장르의 값들을 숫자형으로 바꿔줌.
column_list = u_item.columns[5:]
u_item[column_list] = u_item[column_list].apply(pd.to_numeric)

In [69]:
#u.user -> dataframe으로 변경
u_user = []
with open("./ml-100k/u.user",'r',encoding='UTF-8') as f:
    for line in f.readlines():
        u_user.append(line.strip().split("|")) 
        
user_info = """user id | age | gender | occupation | zip code"""        
        
user_info = user_info.split(" | ")    

u_user = pd.DataFrame(u_user,columns=user_info)

In [71]:
#u.genre ->dataframe으로 변경
genre=[]
with open("./ml-100k/u.genre",'r',encoding='UTF-8') as f:
    for line in f.readlines():
        genre.append(line.strip().split("|")) 
genre = pd.DataFrame(genre,columns=["genre_name","genre_id"]) #어차피 Index와 Genre_ID가 같음 나중에 없애봐

#u.occupation-> dataframe으로 변경
occupation =[]
with open("./ml-100k/u.occupation", 'r',encoding='UTF-8') as f:
    for line in f.readlines():
        occupation.append(line.strip().split("|"))
        
occupation = pd.DataFrame(occupation,columns=["job"])

## 2. Make a list of the top 100 most popular movies

### A-1. Extract a list of movies with the highest number of ratings for all users

In [52]:
# 1.전체이용자에서 가장 시청률이 높은 영화 TOP 100
result = u_data.value_counts(subset="movie id", ascending=False)

result = pd.DataFrame(result)
top_id = result.index[:100] #영화 TOP 100 ID

print("1. 전체 사용자에게 가장 시청률이 높은 영화 TOP 100\n")

for idx in range(100):
    name =  u_item.loc[u_item['movie id']==top_id[idx],"movie title"].values[0]
    print("{0}등 영화 제목 : {1}".format(idx+1,name))


1. 전체 사용자에게 가장 시청률이 높은 영화 TOP 100

1등 영화 제목 : Star Wars (1977)
2등 영화 제목 : Contact (1997)
3등 영화 제목 : Fargo (1996)
4등 영화 제목 : Return of the Jedi (1983)
5등 영화 제목 : Liar Liar (1997)
6등 영화 제목 : English Patient, The (1996)
7등 영화 제목 : Scream (1996)
8등 영화 제목 : Toy Story (1995)
9등 영화 제목 : Air Force One (1997)
10등 영화 제목 : Independence Day (ID4) (1996)
11등 영화 제목 : Raiders of the Lost Ark (1981)
12등 영화 제목 : Godfather, The (1972)
13등 영화 제목 : Pulp Fiction (1994)
14등 영화 제목 : Twelve Monkeys (1995)
15등 영화 제목 : Silence of the Lambs, The (1991)
16등 영화 제목 : Jerry Maguire (1996)
17등 영화 제목 : Rock, The (1996)
18등 영화 제목 : Empire Strikes Back, The (1980)
19등 영화 제목 : Star Trek: First Contact (1996)
20등 영화 제목 : Back to the Future (1985)
21등 영화 제목 : Titanic (1997)
22등 영화 제목 : Mission: Impossible (1996)
23등 영화 제목 : Fugitive, The (1993)
24등 영화 제목 : Indiana Jones and the Last Crusade (1989)
25등 영화 제목 : Willy Wonka and the Chocolate Factory (1971)
26등 영화 제목 : Princess Bride, The (1987)
27등 영화 제목 : Forrest Gump (1994)

### A-2. Extract a list of movies with the highest number of ratings by gender

In [72]:
# 2. gender를 기준으로 시청률이 높은 영화 TOP100

#간소화한 user_data를 만든다. (user_id, gender만 포험함 데이터 셋)
gender_user = u_user.loc[:,["user id","gender"]]

#gender_user와 u_data를 inner joing한다.
join_data = pd.merge(u_data, gender_user)
join_data.head()

#1. 여자만 따로 빼내기
F_join = join_data.loc[join_data['gender']=='F']
F_result = F_join.value_counts(subset="movie id", ascending=False).index[:100]

print("1. 여자 사용자에게 가장 시청률이 높은 영화 TOP 100\n")

for idx in range(100):
    name =  u_item.loc[u_item['movie id']==F_result[idx],"movie title"].values[0]
    print("{0}등 영화 제목 : {1}".format(idx+1,name))
    
#2. 남자만 따로 뺴내기
M_join = join_data.loc[join_data['gender']=='M']
M_result = M_join.value_counts(subset="movie id", ascending=False).index[:100]

print("\n2. 남자 사용자에게 가장 시청률이 높은 영화 TOP 100\n")

for idx in range(100):
    name =  u_item.loc[u_item['movie id']==M_result[idx],"movie title"].values[0]
    print("{0}등 영화 제목 : {1}".format(idx+1,name))

1. 여자 사용자에게 가장 시청률이 높은 영화 TOP 100

1등 영화 제목 : English Patient, The (1996)
2등 영화 제목 : Star Wars (1977)
3등 영화 제목 : Scream (1996)
4등 영화 제목 : Liar Liar (1997)
5등 영화 제목 : Contact (1997)
6등 영화 제목 : Air Force One (1997)
7등 영화 제목 : Fargo (1996)
8등 영화 제목 : Return of the Jedi (1983)
9등 영화 제목 : Toy Story (1995)
10등 영화 제목 : Jerry Maguire (1996)
11등 영화 제목 : Independence Day (ID4) (1996)
12등 영화 제목 : Godfather, The (1972)
13등 영화 제목 : Titanic (1997)
14등 영화 제목 : Silence of the Lambs, The (1991)
15등 영화 제목 : Full Monty, The (1997)
16등 영화 제목 : Truth About Cats & Dogs, The (1996)
17등 영화 제목 : Sense and Sensibility (1995)
18등 영화 제목 : Raiders of the Lost Ark (1981)
19등 영화 제목 : Willy Wonka and the Chocolate Factory (1971)
20등 영화 제목 : Mr. Holland's Opus (1995)
21등 영화 제목 : Empire Strikes Back, The (1980)
22등 영화 제목 : Rock, The (1996)
23등 영화 제목 : Back to the Future (1985)
24등 영화 제목 : Princess Bride, The (1987)
25등 영화 제목 : Birdcage, The (1996)
26등 영화 제목 : When Harry Met Sally... (1989)
27등 영화 제목 : E.T. the Extra-Te

### A-3. Extract a list of movies with the highest number of ratings by occupation

In [73]:
#3. 직업별로 가장 인기있는 영화 TOP 100 출력하기

#간소화한 user_data를 만든다. (user_id, occupation만을 포험함 데이터 셋)
occupation_user = u_user.loc[:,["user id","occupation"]]

#gender_user와 u_data를 inner joing한다.
join_data = pd.merge(u_data, occupation_user)

for i in range(len(occupation)):
    specific_join = join_data.loc[join_data['occupation']==occupation['job'][i]] #occupation별로 데이터 뽑기
    occupation_result = specific_join.value_counts(subset='movie id', ascending=False).index[:100] #시청률 상위 100개 뽑기
    
    print("\n{0}. {1}에게 가장 시청률이 높은 영화 TOP 100\n".format(i+1,occupation['job'][i]))
    
    for idx in range(100):
        name = u_item.loc[u_item['movie id']==occupation_result[idx], "movie title"].values[0] #movie_id를 통해  movie title 추출하기
        print("{0}등 영화 제목 : {1}".format(idx+1, name))
       



1. administrator에게 가장 시청률이 높은 영화 TOP 100

1등 영화 제목 : English Patient, The (1996)
2등 영화 제목 : Star Wars (1977)
3등 영화 제목 : Return of the Jedi (1983)
4등 영화 제목 : Jerry Maguire (1996)
5등 영화 제목 : Air Force One (1997)
6등 영화 제목 : Contact (1997)
7등 영화 제목 : Liar Liar (1997)
8등 영화 제목 : Fargo (1996)
9등 영화 제목 : Silence of the Lambs, The (1991)
10등 영화 제목 : Scream (1996)
11등 영화 제목 : Independence Day (ID4) (1996)
12등 영화 제목 : Godfather, The (1972)
13등 영화 제목 : Full Monty, The (1997)
14등 영화 제목 : Back to the Future (1985)
15등 영화 제목 : Toy Story (1995)
16등 영화 제목 : Pulp Fiction (1994)
17등 영화 제목 : Twelve Monkeys (1995)
18등 영화 제목 : Raiders of the Lost Ark (1981)
19등 영화 제목 : Empire Strikes Back, The (1980)
20등 영화 제목 : Star Trek: First Contact (1996)
21등 영화 제목 : Dead Man Walking (1995)
22등 영화 제목 : Mission: Impossible (1996)
23등 영화 제목 : In & Out (1997)
24등 영화 제목 : Alien (1979)
25등 영화 제목 : Twister (1996)
26등 영화 제목 : To Kill a Mockingbird (1962)
27등 영화 제목 : Birdcage, The (1996)
28등 영화 제목 : Indiana Jones and the Las

62등 영화 제목 : Forrest Gump (1994)
63등 영화 제목 : Casablanca (1942)
64등 영화 제목 : Devil's Own, The (1997)
65등 영화 제목 : Remains of the Day, The (1993)
66등 영화 제목 : Emma (1996)
67등 영화 제목 : Gandhi (1982)
68등 영화 제목 : Field of Dreams (1989)
69등 영화 제목 : Rock, The (1996)
70등 영화 제목 : Jaws (1975)
71등 영화 제목 : Babe (1995)
72등 영화 제목 : Boogie Nights (1997)
73등 영화 제목 : Groundhog Day (1993)
74등 영화 제목 : Secrets & Lies (1996)
75등 영화 제목 : North by Northwest (1959)
76등 영화 제목 : Room with a View, A (1986)
77등 영화 제목 : Blade Runner (1982)
78등 영화 제목 : Tin Cup (1996)
79등 영화 제목 : To Kill a Mockingbird (1962)
80등 영화 제목 : Murder at 1600 (1997)
81등 영화 제목 : GoodFellas (1990)
82등 영화 제목 : Blues Brothers, The (1980)
83등 영화 제목 : Big Night (1996)
84등 영화 제목 : Godfather: Part II, The (1974)
85등 영화 제목 : Twister (1996)
86등 영화 제목 : Annie Hall (1977)
87등 영화 제목 : Phenomenon (1996)
88등 영화 제목 : Glory (1989)
89등 영화 제목 : Terminator, The (1984)
90등 영화 제목 : Wizard of Oz, The (1939)
91등 영화 제목 : Everyone Says I Love You (1996)
92등 영화 제목 : Bridg

79등 영화 제목 : Sense and Sensibility (1995)
80등 영화 제목 : Rainmaker, The (1997)
81등 영화 제목 : Time to Kill, A (1996)
82등 영화 제목 : Primal Fear (1996)
83등 영화 제목 : Jaws (1975)
84등 영화 제목 : Mars Attacks! (1996)
85등 영화 제목 : Shawshank Redemption, The (1994)
86등 영화 제목 : Sound of Music, The (1965)
87등 영화 제목 : Seven Years in Tibet (1997)
88등 영화 제목 : Space Jam (1996)
89등 영화 제목 : Long Kiss Goodnight, The (1996)
90등 영화 제목 : Lone Star (1996)
91등 영화 제목 : Cold Comfort Farm (1995)
92등 영화 제목 : Grumpier Old Men (1995)
93등 영화 제목 : Clear and Present Danger (1994)
94등 영화 제목 : Right Stuff, The (1983)
95등 영화 제목 : Braveheart (1995)
96등 영화 제목 : One Flew Over the Cuckoo's Nest (1975)
97등 영화 제목 : Ben-Hur (1959)
98등 영화 제목 : Absolute Power (1997)
99등 영화 제목 : Deer Hunter, The (1978)
100등 영화 제목 : Annie Hall (1977)

8. healthcare에게 가장 시청률이 높은 영화 TOP 100

1등 영화 제목 : Star Wars (1977)
2등 영화 제목 : Return of the Jedi (1983)
3등 영화 제목 : Air Force One (1997)
4등 영화 제목 : Godfather, The (1972)
5등 영화 제목 : Scream (1996)
6등 영화 제목 : Independ

35등 영화 제목 : Cinema Paradiso (1988)
36등 영화 제목 : Princess Bride, The (1987)
37등 영화 제목 : Apt Pupil (1998)
38등 영화 제목 : Toy Story (1995)
39등 영화 제목 : Monty Python and the Holy Grail (1974)
40등 영화 제목 : Vertigo (1958)
41등 영화 제목 : Lone Star (1996)
42등 영화 제목 : Postino, Il (1994)
43등 영화 제목 : Forrest Gump (1994)
44등 영화 제목 : 2001: A Space Odyssey (1968)
45등 영화 제목 : Rock, The (1996)
46등 영화 제목 : Back to the Future (1985)
47등 영화 제목 : Liar Liar (1997)
48등 영화 제목 : Raising Arizona (1987)
49등 영화 제목 : Fish Called Wanda, A (1988)
50등 영화 제목 : To Kill a Mockingbird (1962)
51등 영화 제목 : As Good As It Gets (1997)
52등 영화 제목 : Groundhog Day (1993)
53등 영화 제목 : Independence Day (ID4) (1996)
54등 영화 제목 : Rear Window (1954)
55등 영화 제목 : Usual Suspects, The (1995)
56등 영화 제목 : Willy Wonka and the Chocolate Factory (1971)
57등 영화 제목 : Braveheart (1995)
58등 영화 제목 : Good Will Hunting (1997)
59등 영화 제목 : Boogie Nights (1997)
60등 영화 제목 : Ulee's Gold (1997)
61등 영화 제목 : Twelve Monkeys (1995)
62등 영화 제목 : Boot, Das (1981)
63등 영화 제목 :

56등 영화 제목 : Lion King, The (1994)
57등 영화 제목 : Dead Poets Society (1989)
58등 영화 제목 : Braveheart (1995)
59등 영화 제목 : Raising Arizona (1987)
60등 영화 제목 : L.A. Confidential (1997)
61등 영화 제목 : Monty Python and the Holy Grail (1974)
62등 영화 제목 : Indiana Jones and the Last Crusade (1989)
63등 영화 제목 : Game, The (1997)
64등 영화 제목 : George of the Jungle (1997)
65등 영화 제목 : Aladdin (1992)
66등 영화 제목 : Time to Kill, A (1996)
67등 영화 제목 : Jaws (1975)
68등 영화 제목 : Amadeus (1984)
69등 영화 제목 : Trainspotting (1996)
70등 영화 제목 : Chasing Amy (1997)
71등 영화 제목 : Primal Fear (1996)
72등 영화 제목 : Sleepless in Seattle (1993)
73등 영화 제목 : One Flew Over the Cuckoo's Nest (1975)
74등 영화 제목 : Casablanca (1942)
75등 영화 제목 : Beauty and the Beast (1991)
76등 영화 제목 : Emma (1996)
77등 영화 제목 : Aliens (1986)
78등 영화 제목 : Wizard of Oz, The (1939)
79등 영화 제목 : Fish Called Wanda, A (1988)
80등 영화 제목 : Heat (1995)
81등 영화 제목 : Seven (Se7en) (1995)
82등 영화 제목 : Psycho (1960)
83등 영화 제목 : Usual Suspects, The (1995)
84등 영화 제목 : Star Trek: The Wrath o

52등 영화 제목 : Sting, The (1973)
53등 영화 제목 : Star Trek: First Contact (1996)
54등 영화 제목 : Monty Python's Life of Brian (1979)
55등 영화 제목 : Nine Months (1995)
56등 영화 제목 : When Harry Met Sally... (1989)
57등 영화 제목 : Field of Dreams (1989)
58등 영화 제목 : Indiana Jones and the Last Crusade (1989)
59등 영화 제목 : Four Weddings and a Funeral (1994)
60등 영화 제목 : Shining, The (1980)
61등 영화 제목 : Back to the Future (1985)
62등 영화 제목 : Fugitive, The (1993)
63등 영화 제목 : Beavis and Butt-head Do America (1996)
64등 영화 제목 : Tin Cup (1996)
65등 영화 제목 : Sleepless in Seattle (1993)
66등 영화 제목 : Booty Call (1997)
67등 영화 제목 : Crash (1996)
68등 영화 제목 : Sense and Sensibility (1995)
69등 영화 제목 : Full Monty, The (1997)
70등 영화 제목 : L.A. Confidential (1997)
71등 영화 제목 : That Thing You Do! (1996)
72등 영화 제목 : Boogie Nights (1997)
73등 영화 제목 : Mrs. Doubtfire (1993)
74등 영화 제목 : Truth About Cats & Dogs, The (1996)
75등 영화 제목 : Home Alone (1990)
76등 영화 제목 : Midnight in the Garden of Good and Evil (1997)
77등 영화 제목 : Jackie Brown (1997)
78등 영

### B-1. Extract a list of movies with the highest average ratings for all users

In [55]:
#전체 데이터에서 각 movie별로 평균점수 뽑기
all_mean = u_data.groupby(['movie id']).mean()
all_mean = all_mean.sort_values(by=['rating'],ascending=False)[:100]

print("1. 전체 사용자에게 가장 높은 평점을 받은 영화 TOP 100\n")

for idx in range(100):
    name = u_item.loc[u_item['movie id']==all_mean.index[idx], "movie title"].values[0] #movie_id를 통해  movie title 추출하기
    print("{0}등 영화 제목 : {1}".format(idx+1, name))
       

1. 전체 사용자에게 가장 높은 평점을 받은 영화 TOP 100

1등 영화 제목 : Santa with Muscles (1996)
2등 영화 제목 : Star Kid (1997)
3등 영화 제목 : Prefontaine (1997)
4등 영화 제목 : Entertaining Angels: The Dorothy Day Story (1996)
5등 영화 제목 : Saint of Fort Washington, The (1993)
6등 영화 제목 : They Made Me a Criminal (1939)
7등 영화 제목 : Someone Else's America (1995)
8등 영화 제목 : Marlene Dietrich: Shadow and Light (1996) 
9등 영화 제목 : Aiqing wansui (1994)
10등 영화 제목 : Great Day in Harlem, A (1994)
11등 영화 제목 : Pather Panchali (1955)
12등 영화 제목 : Anna (1996)
13등 영화 제목 : Some Mother's Son (1996)
14등 영화 제목 : Maya Lin: A Strong Clear Vision (1994)
15등 영화 제목 : Everest (1998)
16등 영화 제목 : Close Shave, A (1995)
17등 영화 제목 : Schindler's List (1993)
18등 영화 제목 : Wrong Trousers, The (1993)
19등 영화 제목 : Casablanca (1942)
20등 영화 제목 : Wallace & Gromit: The Best of Aardman Animation (1996)
21등 영화 제목 : Shawshank Redemption, The (1994)
22등 영화 제목 : Rear Window (1954)
23등 영화 제목 : Usual Suspects, The (1995)
24등 영화 제목 : Star Wars (1977)
25등 영화 제목 : 12 Angry Men 

### B-2. Extract a list of movies with the highest average ratings by gender

In [74]:
# 2. gender를 기준으로 나눴을 때 평점이 높은 영화 TOP100

#간소화한 user_data를 만든다. (user_id, gender만 포험함 데이터 셋)
gender_user = u_user.loc[:,["user id","gender"]]

#gender_user와 u_data를 inner joing한다.
join_data = pd.merge(u_data, gender_user)

#1. 여자만 따로 빼내기
F_join = join_data.loc[join_data['gender']=='F']
F_join = F_join.groupby(['movie id']).mean() #각 영화별 평균평점구하기
F_result = F_join.sort_values(by=['rating'], ascending=False).index[:100]

print("1. 여자 사용자에게 평점이 높은 영화 TOP 100\n")

for idx in range(100):
    name =  u_item.loc[u_item['movie id']==F_result[idx],"movie title"].values[0]
    print("{0}등 영화 제목 : {1}".format(idx+1,name))
    
    
#2. 남자만 따로 뺴내기
M_join = join_data.loc[join_data['gender']=='M']
M_join = M_join.groupby(['movie id']).mean()
M_result = M_join.sort_values(by=['rating'], ascending=False).index[:100]

print("\n2. 남자 사용자에게 가장 시청률이 높은 영화 TOP 100\n")

for idx in range(100):
    name =  u_item.loc[u_item['movie id']==M_result[idx],"movie title"].values[0]
    print("{0}등 영화 제목 : {1}".format(idx+1,name))

1. 여자 사용자에게 평점이 높은 영화 TOP 100

1등 영화 제목 : Everest (1998)
2등 영화 제목 : Visitors, The (Visiteurs, Les) (1993)
3등 영화 제목 : Someone Else's America (1995)
4등 영화 제목 : Maya Lin: A Strong Clear Vision (1994)
5등 영화 제목 : Foreign Correspondent (1940)
6등 영화 제목 : Faster Pussycat! Kill! Kill! (1965)
7등 영화 제목 : Mina Tannenbaum (1994)
8등 영화 제목 : Prefontaine (1997)
9등 영화 제목 : Telling Lies in America (1997)
10등 영화 제목 : Year of the Horse (1997)
11등 영화 제목 : Stripes (1981)
12등 영화 제목 : Schindler's List (1993)
13등 영화 제목 : Close Shave, A (1995)
14등 영화 제목 : Shawshank Redemption, The (1994)
15등 영화 제목 : Wallace & Gromit: The Best of Aardman Animation (1996)
16등 영화 제목 : Shall We Dance? (1996)
17등 영화 제목 : Friday (1995)
18등 영화 제목 : Women, The (1939)
19등 영화 제목 : Some Mother's Son (1996)
20등 영화 제목 : Band Wagon, The (1953)
21등 영화 제목 : Wrong Trousers, The (1993)
22등 영화 제목 : Some Folks Call It a Sling Blade (1993)
23등 영화 제목 : Sling Blade (1996)
24등 영화 제목 : Secrets & Lies (1996)
25등 영화 제목 : Casablanca (1942)
26등 영화 제목 : Chr

### B-3. Extract a list of movies with the highest average ratings by occupation

In [77]:
#3. 직업별로 평균 평점이 높은 TOP 100


#간소화한 user_data를 만든다. (user_id, occupation만을 포험함 데이터 셋)
occupation_user = u_user.loc[:,["user id","occupation"]]

#gender_user와 u_data를 inner joing한다.
join_data = pd.merge(u_data, occupation_user)

for i in range(len(occupation)):
    specific_join = join_data.loc[join_data['occupation']==occupation['job'][i]] #occupation별로 데이터 뽑기
    specific_join = specific_join.groupby(['movie id']).mean()
    occupation_result = specific_join.sort_values(by='rating', ascending=False).index[:100] #시청률 상위 100개 뽑기
    
    print("\n{0}. {1}에게 평균 평점이 높은 영화 TOP 100\n".format(i+1,occupation['job'][i]))
    
    for idx in range(100):
        name = u_item.loc[u_item['movie id']==occupation_result[idx], "movie title"].values[0] #movie_id를 통해  movie title 추출하기
        print("{0}등 영화 제목 : {1}".format(idx+1, name))
       




1. administrator에게 평균 평점이 높은 영화 TOP 100

1등 영화 제목 : Audrey Rose (1977)
2등 영화 제목 : Tango Lesson, The (1997)
3등 영화 제목 : Paradise Road (1997)
4등 영화 제목 : Prefontaine (1997)
5등 영화 제목 : Raise the Red Lantern (1991)
6등 영화 제목 : Paradise Lost: The Child Murders at Robin Hood Hills (1996)
7등 영화 제목 : Boys of St. Vincent, The (1993)
8등 영화 제목 : Congo (1995)
9등 영화 제목 : Assignment, The (1997)
10등 영화 제목 : Close Shave, A (1995)
11등 영화 제목 : Winter Guest, The (1997)
12등 영화 제목 : Once Were Warriors (1994)
13등 영화 제목 : NeverEnding Story III, The (1994)
14등 영화 제목 : Cry, the Beloved Country (1995)
15등 영화 제목 : Bread and Chocolate (Pane e cioccolata) (1973)
16등 영화 제목 : Ponette (1996)
17등 영화 제목 : Chairman of the Board (1998)
18등 영화 제목 : Bogus (1996)
19등 영화 제목 : Gay Divorcee, The (1934)
20등 영화 제목 : Grass Harp, The (1995)
21등 영화 제목 : Selena (1997)
22등 영화 제목 : Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)
23등 영화 제목 : Braindead (1992)
24등 영화 제목 : A Chef in Love (1996)
25등 영화 제목 : Four Days in Sep


4. educator에게 평균 평점이 높은 영화 TOP 100

1등 영화 제목 : Marlene Dietrich: Shadow and Light (1996) 
2등 영화 제목 : Boy's Life 2 (1997)
3등 영화 제목 : Infinity (1996)
4등 영화 제목 : Unstrung Heroes (1995)
5등 영화 제목 : Stripes (1981)
6등 영화 제목 : Incognito (1997)
7등 영화 제목 : Night on Earth (1991)
8등 영화 제목 : Cemetery Man (Dellamorte Dellamore) (1994)
9등 영화 제목 : Purple Noon (1960)
10등 영화 제목 : Strawberry and Chocolate (Fresa y chocolate) (1993)
11등 영화 제목 : Maya Lin: A Strong Clear Vision (1994)
12등 영화 제목 : Stalker (1979)
13등 영화 제목 : Four Days in September (1997)
14등 영화 제목 : Go Fish (1994)
15등 영화 제목 : Steel (1997)
16등 영화 제목 : War Room, The (1993)
17등 영화 제목 : Baby-Sitters Club, The (1995)
18등 영화 제목 : Crooklyn (1994)
19등 영화 제목 : Faithful (1996)
20등 영화 제목 : Double Happiness (1994)
21등 영화 제목 : Hearts and Minds (1996)
22등 영화 제목 : Great Day in Harlem, A (1994)
23등 영화 제목 : Whole Wide World, The (1996)
24등 영화 제목 : For the Moment (1994)
25등 영화 제목 : Two or Three Things I Know About Her (1966)
26등 영화 제목 : Pather Panchali (1955)

73등 영화 제목 : Mighty Aphrodite (1995)
74등 영화 제목 : Spice World (1997)
75등 영화 제목 : Waiting for Guffman (1996)
76등 영화 제목 : Casino (1995)
77등 영화 제목 : Alien (1979)
78등 영화 제목 : Heathers (1989)
79등 영화 제목 : Cook the Thief His Wife & Her Lover, The (1989)
80등 영화 제목 : Gridlock'd (1997)
81등 영화 제목 : Forrest Gump (1994)
82등 영화 제목 : African Queen, The (1951)
83등 영화 제목 : Nikita (La Femme Nikita) (1990)
84등 영화 제목 : When Harry Met Sally... (1989)
85등 영화 제목 : Raising Arizona (1987)
86등 영화 제목 : Sling Blade (1996)
87등 영화 제목 : Bad Taste (1987)
88등 영화 제목 : Apt Pupil (1998)
89등 영화 제목 : Freeway (1996)
90등 영화 제목 : Third Man, The (1949)
91등 영화 제목 : Boot, Das (1981)
92등 영화 제목 : Fried Green Tomatoes (1991)
93등 영화 제목 : Apollo 13 (1995)
94등 영화 제목 : North by Northwest (1959)
95등 영화 제목 : Wallace & Gromit: The Best of Aardman Animation (1996)
96등 영화 제목 : It's a Wonderful Life (1946)
97등 영화 제목 : Big Sleep, The (1946)
98등 영화 제목 : Die xue shuang xiong (Killer, The) (1989)
99등 영화 제목 : Breaking the Waves (1996)
100등 영화 제목 : 

84등 영화 제목 : Tomorrow Never Dies (1997)
85등 영화 제목 : Saint, The (1997)
86등 영화 제목 : Truth About Cats & Dogs, The (1996)
87등 영화 제목 : Birdcage, The (1996)
88등 영화 제목 : Mr. Holland's Opus (1995)
89등 영화 제목 : Mission: Impossible (1996)
90등 영화 제목 : Michael (1996)
91등 영화 제목 : Midnight in the Garden of Good and Evil (1997)
92등 영화 제목 : Scream (1996)
93등 영화 제목 : Conspiracy Theory (1997)
94등 영화 제목 : Chain Reaction (1996)
95등 영화 제목 : Smile Like Yours, A (1997)
96등 영화 제목 : Looking for Richard (1996)
97등 영화 제목 : Dangerous Ground (1997)
98등 영화 제목 : Godfather, The (1972)
99등 영화 제목 : Silence of the Lambs, The (1991)
100등 영화 제목 : Mouse Hunt (1997)

10. lawyer에게 평균 평점이 높은 영화 TOP 100

1등 영화 제목 : Groundhog Day (1993)
2등 영화 제목 : Rear Window (1954)
3등 영화 제목 : Shine (1996)
4등 영화 제목 : Anastasia (1997)
5등 영화 제목 : Interview with the Vampire (1994)
6등 영화 제목 : Usual Suspects, The (1995)
7등 영화 제목 : Quick and the Dead, The (1995)
8등 영화 제목 : Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)
9등 영화 제목 : W

51등 영화 제목 : Great Escape, The (1963)
52등 영화 제목 : Boot, Das (1981)
53등 영화 제목 : Gone with the Wind (1939)
54등 영화 제목 : Amadeus (1984)
55등 영화 제목 : Patton (1970)
56등 영화 제목 : Mr. Smith Goes to Washington (1939)
57등 영화 제목 : Good Will Hunting (1997)
58등 영화 제목 : Apocalypse Now (1979)
59등 영화 제목 : Night of the Living Dead (1968)
60등 영화 제목 : Lawrence of Arabia (1962)
61등 영화 제목 : Christmas Carol, A (1938)
62등 영화 제목 : Sword in the Stone, The (1963)
63등 영화 제목 : Cinema Paradiso (1988)
64등 영화 제목 : Dial M for Murder (1954)
65등 영화 제목 : Hoodlum (1997)
66등 영화 제목 : To Kill a Mockingbird (1962)
67등 영화 제목 : Kundun (1997)
68등 영화 제목 : Ice Storm, The (1997)
69등 영화 제목 : Cool Hand Luke (1967)
70등 영화 제목 : For Whom the Bell Tolls (1943)
71등 영화 제목 : Dave (1993)
72등 영화 제목 : When Harry Met Sally... (1989)
73등 영화 제목 : Godfather, The (1972)
74등 영화 제목 : Titanic (1997)
75등 영화 제목 : Glory (1989)
76등 영화 제목 : Aliens (1986)
77등 영화 제목 : Bridge on the River Kwai, The (1957)
78등 영화 제목 : Alien (1979)
79등 영화 제목 : Great Dictator, The

32등 영화 제목 : Star Wars (1977)
33등 영화 제목 : Once Upon a Time in the West (1969)
34등 영화 제목 : Paris Is Burning (1990)
35등 영화 제목 : Shadowlands (1993)
36등 영화 제목 : Affair to Remember, An (1957)
37등 영화 제목 : Return of Martin Guerre, The (Retour de Martin Guerre, Le) (1982)
38등 영화 제목 : Dazed and Confused (1993)
39등 영화 제목 : Big Lebowski, The (1998)
40등 영화 제목 : Shall We Dance? (1937)
41등 영화 제목 : Ulee's Gold (1997)
42등 영화 제목 : Mallrats (1995)
43등 영화 제목 : My Man Godfrey (1936)
44등 영화 제목 : Die xue shuang xiong (Killer, The) (1989)
45등 영화 제목 : Eve's Bayou (1997)
46등 영화 제목 : Prophecy, The (1995)
47등 영화 제목 : High Noon (1952)
48등 영화 제목 : Love! Valour! Compassion! (1997)
49등 영화 제목 : Flirting With Disaster (1996)
50등 영화 제목 : Sunset Blvd. (1950)
51등 영화 제목 : White Squall (1996)
52등 영화 제목 : Sleeper (1973)
53등 영화 제목 : 20,000 Leagues Under the Sea (1954)
54등 영화 제목 : Basketball Diaries, The (1995)
55등 영화 제목 : Mary Shelley's Frankenstein (1994)
56등 영화 제목 : Koyaanisqatsi (1983)
57등 영화 제목 : One Flew Over the Cuckoo'

93등 영화 제목 : African Queen, The (1951)
94등 영화 제목 : Star Wars (1977)
95등 영화 제목 : Twelve Monkeys (1995)
96등 영화 제목 : Braveheart (1995)
97등 영화 제목 : Terminator, The (1984)
98등 영화 제목 : Boot, Das (1981)
99등 영화 제목 : Star Trek: First Contact (1996)
100등 영화 제목 : Shining, The (1980)

19. student에게 평균 평점이 높은 영화 TOP 100

1등 영화 제목 : Cement Garden, The (1993)
2등 영화 제목 : Faust (1994)
3등 영화 제목 : Paradise Lost: The Child Murders at Robin Hood Hills (1996)
4등 영화 제목 : Wedding Gift, The (1994)
5등 영화 제목 : Critical Care (1997)
6등 영화 제목 : Before the Rain (Pred dozhdot) (1994)
7등 영화 제목 : Wild Things (1998)
8등 영화 제목 : Dream With the Fishes (1997)
9등 영화 제목 : Inspector General, The (1949)
10등 영화 제목 : Prefontaine (1997)
11등 영화 제목 : Saint of Fort Washington, The (1993)
12등 영화 제목 : Stonewall (1995)
13등 영화 제목 : Once Upon a Time... When We Were Colored (1995)
14등 영화 제목 : Shiloh (1997)
15등 영화 제목 : Santa with Muscles (1996)
16등 영화 제목 : Giant (1956)
17등 영화 제목 : Underground (1995)
18등 영화 제목 : Horse Whisperer, The (1998)
19

19등 영화 제목 : Mrs. Dalloway (1997)
20등 영화 제목 : Welcome To Sarajevo (1997)
21등 영화 제목 : Twisted (1996)
22등 영화 제목 : Some Folks Call It a Sling Blade (1993)
23등 영화 제목 : Love in the Afternoon (1957)
24등 영화 제목 : Romper Stomper (1992)
25등 영화 제목 : Kaspar Hauser (1993)
26등 영화 제목 : Psycho (1960)
27등 영화 제목 : Raging Bull (1980)
28등 영화 제목 : Philadelphia Story, The (1940)
29등 영화 제목 : Roman Holiday (1953)
30등 영화 제목 : Kundun (1997)
31등 영화 제목 : Chinatown (1974)
32등 영화 제목 : Sling Blade (1996)
33등 영화 제목 : Apartment, The (1960)
34등 영화 제목 : Braindead (1992)
35등 영화 제목 : Anne Frank Remembered (1995)
36등 영화 제목 : When We Were Kings (1996)
37등 영화 제목 : Man Who Knew Too Little, The (1997)
38등 영화 제목 : Wings of Desire (1987)
39등 영화 제목 : Daylight (1996)
40등 영화 제목 : Local Hero (1983)
41등 영화 제목 : Alphaville (1965)
42등 영화 제목 : Dolores Claiborne (1994)
43등 영화 제목 : Top Hat (1935)
44등 영화 제목 : Mrs. Winterbourne (1996)
45등 영화 제목 : Die xue shuang xiong (Killer, The) (1989)
46등 영화 제목 : Belle de jour (1967)
47등 영화 제목 : Chungking

## 3. Calculate the ratings by the genre

### A-1. ### B-2. Count the number of ratings by each genre for all users and sort them in descending order respectively

In [78]:
# 전체 사용자에게 시청률이 가장 높은 장르를 세고 count해서 내림차순하여라(큰거 -> 작은거)

#u_item과 u_data를 inner joing (on movie id)한다.
join_data = pd.merge(u_data, u_item)
genre_data = []

for i in range(len(genre)-1):
    temp = []
    total = join_data[genre['genre_name'][i]].sum()
    temp.append(genre['genre_name'][i])
    temp.append(total)
    genre_data.append(temp)    
    
genre_data = pd.DataFrame(genre_data,columns=['genre_name','count'])


genre_data = genre_data.sort_values(by='count', ascending=False).reset_index(drop=True)

print("\n전체 사용자가 가장 많이 본 장르를 내림차순으로\n")

for idx in range(len(genre_data)):
    print("{0}등 영화 장르 : {1}, {2}".format(idx+1, genre_data['genre_name'][idx],genre_data['count'][idx]))### B-2. Count the number of ratings by each genre for genders and sort them in descending order respectively


전체 사용자가 가장 많이 본 장르를 내림차순으로

1등 영화 장르 : Drama, 39895
2등 영화 장르 : Comedy, 29832
3등 영화 장르 : Action, 25589
4등 영화 장르 : Thriller, 21872
5등 영화 장르 : Romance, 19461
6등 영화 장르 : Adventure, 13753
7등 영화 장르 : Sci-Fi, 12730
8등 영화 장르 : War, 9398
9등 영화 장르 : Crime, 8055
10등 영화 장르 : Children's, 7182
11등 영화 장르 : Horror, 5317
12등 영화 장르 : Mystery, 5245
13등 영화 장르 : Musical, 4954
14등 영화 장르 : Animation, 3605
15등 영화 장르 : Western, 1854
16등 영화 장르 : Film-Noir, 1733
17등 영화 장르 : Fantasy, 1352
18등 영화 장르 : Documentary, 758
19등 영화 장르 : unknown, 10


### B-2. Count the number of ratings by each genre for genders and sort them in descending order respectively

In [59]:
# 성별로 시청률이 가장 높은 장르를 count해서 내림차순하여라(큰거 -> 작은거)

#u_item과 u_data를 inner joing (on movie id)한다.
join_data = pd.merge(u_data, u_item)

#gender_user와 u_data를 inner joing한다.
gender_user = u_user.loc[:,["user id","gender"]]
join_data = pd.merge(join_data, gender_user)

#1. 여자만 따로 빼내기
F_join = join_data.loc[join_data['gender']=='F']

genre_data = []

for i in range(len(genre)-1):
    temp = []
    total = F_join[genre['genre_name'][i]].sum()
    temp.append(genre['genre_name'][i])
    temp.append(total)
    genre_data.append(temp)   
    #

genre_data = pd.DataFrame(genre_data,columns=['genre_name','count'])
genre_data = genre_data.sort_values(by='count', ascending=False).reset_index(drop=True)

print("\n여자 사용자가 가장 많이 본 장르를 내림차순으로\n")

for idx in range(len(genre_data)):
    print("{0}등 영화 장르 : {1}, {2}".format(idx+1, genre_data['genre_name'][idx],genre_data['count'][idx]))    
    

#2. 남자만 따로 뺴내기
M_join = join_data.loc[join_data['gender']=='M']

genre_data = []

for i in range(len(genre)-1):
    temp = []
    total = M_join[genre['genre_name'][i]].sum()
    temp.append(genre['genre_name'][i])
    temp.append(total)
    genre_data.append(temp)    

    
genre_data = pd.DataFrame(genre_data,columns=['genre_name','count'])
genre_data = genre_data.sort_values(by='count', ascending=False).reset_index(drop=True)

print("\n남자 사용자가 가장 많이 본 장르를 내림차순으로\n")

for idx in range(len(genre_data)):
    print("{0}등 영화 장르 : {1}, {2}".format(idx+1, genre_data['genre_name'][idx],genre_data['count'][idx]))    
   


여자 사용자가 가장 많이 본 장르를 내림차순으로

1등 영화 장르 : Drama, 11008
2등 영화 장르 : Comedy, 8068
3등 영화 장르 : Romance, 5858
4등 영화 장르 : Action, 5442
5등 영화 장르 : Thriller, 5086
6등 영화 장르 : Adventure, 3141
7등 영화 장르 : Sci-Fi, 2629
8등 영화 장르 : Children's, 2232
9등 영화 장르 : War, 2189
10등 영화 장르 : Crime, 1794
11등 영화 장르 : Musical, 1442
12등 영화 장르 : Mystery, 1314
13등 영화 장르 : Horror, 1197
14등 영화 장르 : Animation, 995
15등 영화 장르 : Film-Noir, 385
16등 영화 장르 : Western, 371
17등 영화 장르 : Fantasy, 363
18등 영화 장르 : Documentary, 187
19등 영화 장르 : unknown, 2

남자 사용자가 가장 많이 본 장르를 내림차순으로

1등 영화 장르 : Drama, 28887
2등 영화 장르 : Comedy, 21764
3등 영화 장르 : Action, 20147
4등 영화 장르 : Thriller, 16786
5등 영화 장르 : Romance, 13603
6등 영화 장르 : Adventure, 10612
7등 영화 장르 : Sci-Fi, 10101
8등 영화 장르 : War, 7209
9등 영화 장르 : Crime, 6261
10등 영화 장르 : Children's, 4950
11등 영화 장르 : Horror, 4120
12등 영화 장르 : Mystery, 3931
13등 영화 장르 : Musical, 3512
14등 영화 장르 : Animation, 2610
15등 영화 장르 : Western, 1483
16등 영화 장르 : Film-Noir, 1348
17등 영화 장르 : Fantasy, 989
18등 영화 장르 : Documentary, 

### A-3. Count the number of ratings by each genre for all occupations  and sort them in descending order respectively

In [60]:
# 직업별로 장르를 본 회수를 count해서 내림차순하여라(큰거 -> 작은거)

#u_item과 u_data를 inner joing (on movie id)한다.
join_data = pd.merge(u_data, u_item)

#간소화한 user_data를 만든다. (user_id, occupation만을 포험함 데이터 셋)
occupation_user = u_user.loc[:,["user id","occupation"]]

#gender_user와 join_data inner joing한다.
join_data = pd.merge(join_data, occupation_user)

for idx in range(len(occupation)):
    occupation_data = join_data.loc[join_data['occupation']==occupation['job'][idx]] #occupation별로 데이터 뽑기
    
    genre_data = []
    for i in range(len(genre)-1):
        temp = []
        total = occupation_data[genre['genre_name'][i]].sum()
        temp.append(genre['genre_name'][i])
        temp.append(total)
        genre_data.append(temp)    

    genre_data = pd.DataFrame(genre_data,columns=['genre_name','count'])
    genre_data = genre_data.sort_values(by='count', ascending=False).reset_index(drop=True)
    
    print("\n{0}. {1}이 가장 많이 본 장르\n".format(idx+1,occupation['job'][idx]))
    
    for i in range(len(genre_data)):
        print("{0}등 영화 장르 : {1}, {2}".format(i+1, genre_data['genre_name'][i],genre_data['count'][i]))




1. administrator이 가장 많이 본 장르

1등 영화 장르 : Drama, 3099
2등 영화 장르 : Comedy, 2203
3등 영화 장르 : Action, 1858
4등 영화 장르 : Thriller, 1570
5등 영화 장르 : Romance, 1539
6등 영화 장르 : Adventure, 1020
7등 영화 장르 : Sci-Fi, 942
8등 영화 장르 : War, 786
9등 영화 장르 : Crime, 565
10등 영화 장르 : Children's, 462
11등 영화 장르 : Horror, 391
12등 영화 장르 : Mystery, 385
13등 영화 장르 : Musical, 354
14등 영화 장르 : Animation, 203
15등 영화 장르 : Western, 168
16등 영화 장르 : Film-Noir, 147
17등 영화 장르 : Fantasy, 94
18등 영화 장르 : Documentary, 48
19등 영화 장르 : unknown, 0

2. artist이 가장 많이 본 장르

1등 영화 장르 : Drama, 957
2등 영화 장르 : Comedy, 617
3등 영화 장르 : Action, 528
4등 영화 장르 : Thriller, 476
5등 영화 장르 : Romance, 462
6등 영화 장르 : Adventure, 311
7등 영화 장르 : Sci-Fi, 308
8등 영화 장르 : War, 225
9등 영화 장르 : Children's, 202
10등 영화 장르 : Crime, 201
11등 영화 장르 : Mystery, 164
12등 영화 장르 : Musical, 157
13등 영화 장르 : Animation, 121
14등 영화 장르 : Horror, 102
15등 영화 장르 : Film-Noir, 64
16등 영화 장르 : Documentary, 34
17등 영화 장르 : Western, 29
18등 영화 장르 : Fantasy, 24
19등 영화 장르 : unknown, 0

3. doctor이 가

### B-1. Measure the average ratings by genre for all users and sort them in descending order respectively

In [81]:
# 전체 사용자가 평가한 영화 장르의 평점의 평균을 구한 후 내림차순하여라(큰거 -> 작은거)

#u_item과 u_data를 inner joing (on movie id)한다.
join_data = pd.merge(u_data, u_item)
#genre_data에 장르와 평점의 평균을 저장한다.
genre_data = []

for i in range(len(genre)-1):
    temp = []
    #현재 다루는 장르
    current_genre = genre['genre_name'][i]
    #현재 장르를 평가한 데이터를 뽑아낸다.
    specific_genre = join_data.loc[join_data[current_genre]==1,[current_genre,'rating']]
    genre_mean = specific_genre['rating'].mean()
    temp.append(current_genre)
    temp.append(genre_mean)
    genre_data.append(temp)

    
genre_data = pd.DataFrame(genre_data,columns=['genre_name','mean'])
#평점을 기준으로 정렬
genre_data = genre_data.sort_values(by='mean', ascending=False).reset_index(drop=True)

print("\n전체 사용자가 평가한 영화 장르 평균 평점을 내림차순으로\n")

for idx in range(len(genre_data)):
    print("{0}등 영화 장르 : {1}, {2}".format(idx+1, genre_data['genre_name'][idx],genre_data['mean'][idx]))



전체 사용자가 평가한 영화 장르 평균 평점을 내림차순으로

1등 영화 장르 : Film-Noir, 3.9215233698788228
2등 영화 장르 : War, 3.815811874866993
3등 영화 장르 : Drama, 3.6873793708484772
4등 영화 장르 : Documentary, 3.6728232189973613
5등 영화 장르 : Mystery, 3.63813155386082
6등 영화 장르 : Crime, 3.6322780881440098
7등 영화 장르 : Romance, 3.621704948358255
8등 영화 장르 : Western, 3.6132686084142396
9등 영화 장르 : Animation, 3.5766990291262135
10등 영화 장르 : Sci-Fi, 3.5607227022780834
11등 영화 장르 : Musical, 3.521396851029471
12등 영화 장르 : Thriller, 3.5090069495245064
13등 영화 장르 : Adventure, 3.503526503308369
14등 영화 장르 : Action, 3.480245417953027
15등 영화 장르 : Comedy, 3.3940734781442745
16등 영화 장르 : Children's, 3.3532442216652742
17등 영화 장르 : Horror, 3.2903893172841827
18등 영화 장르 : Fantasy, 3.2152366863905324
19등 영화 장르 : unknown, 3.2


### B-2. Measure the average ratings by genre for genders and sort them in descending order respectively

In [83]:
#남녀 사용자가 평가한 영화 장르의 평점의 평균을 구한 후 내림차순하여라(큰거 -> 작은거)

#u_item과 u_data를 inner joing (on movie id)한다.
join_data = pd.merge(u_data, u_item)

#gender_user와 join_dta를 inner joing한다.
gender_user = u_user.loc[:,["user id","gender"]]
join_data = pd.merge(join_data, gender_user)

#사용자 중 여자의 데이터만 뽑아낸다.
F_join = join_data.loc[join_data['gender']=='F']

#genre_data에 장르와 평점의 평균을 저장한다.
genre_data = []
for i in range(len(genre)-1):
    temp = []
    #현재 다루는 장르
    current_genre = genre['genre_name'][i]
    #현재 장르를 평가한 데이터를 뽑아낸다.
    specific_genre = F_join.loc[F_join[current_genre]==1,[current_genre,'rating']]
    genre_mean = specific_genre['rating'].mean()
    temp.append(current_genre)
    temp.append(genre_mean)
    genre_data.append(temp)

    
genre_data = pd.DataFrame(genre_data,columns=['genre_name','mean'])
#평점을 기준으로 정렬
genre_data = genre_data.sort_values(by='mean', ascending=False).reset_index(drop=True)

print("\n여자 사용자가 평가한 영화 장르의 평균 평점을 내림차순으로\n")

for idx in range(len(genre_data)):
    print("{0}등 영화 장르 : {1}, {2}".format(idx+1, genre_data['genre_name'][idx],genre_data['mean'][idx]))

    
 #사용자 중 남자의 데이터만 뽑아낸다.
M_join = join_data.loc[join_data['gender']=='M']

#genre_data에 장르와 평점의 평균을 저장한다.
genre_data = []
for i in range(len(genre)-1):
    temp = []
    #현재 다루는 장르
    current_genre = genre['genre_name'][i]
    #현재 장르를 평가한 데이터를 뽑아낸다.
    specific_genre = M_join.loc[M_join[current_genre]==1,[current_genre,'rating']]
    genre_mean = specific_genre['rating'].mean()
    temp.append(current_genre)
    temp.append(genre_mean)
    genre_data.append(temp)

    
genre_data = pd.DataFrame(genre_data,columns=['genre_name','mean'])
#평점을 기준으로 정렬
genre_data = genre_data.sort_values(by='mean', ascending=False).reset_index(drop=True)

print("\n남자 사용자가 평가한 영화 장르의 평균 평점을 내림차순으로\n")

for idx in range(len(genre_data)):
    print("{0}등 영화 장르 : {1}, {2}".format(idx+1, genre_data['genre_name'][idx],genre_data['mean'][idx]))   


여자 사용자가 평가한 영화 장르의 평균 평점을 내림차순으로

1등 영화 장르 : War, 3.7811786203746003
2등 영화 장르 : Film-Noir, 3.74025974025974
3등 영화 장르 : Drama, 3.6622456395348837
4등 영화 장르 : Romance, 3.6556845339706383
5등 영화 장르 : Musical, 3.6400832177531206
6등 영화 장르 : Animation, 3.62713567839196
7등 영화 장르 : Documentary, 3.6149732620320854
8등 영화 장르 : Mystery, 3.5601217656012176
9등 영화 장르 : Crime, 3.556298773690078
10등 영화 장르 : Adventure, 3.517987901942057
11등 영화 장르 : Western, 3.5148247978436657
12등 영화 장르 : unknown, 3.5
13등 영화 장르 : Sci-Fi, 3.497907949790795
14등 영화 장르 : Thriller, 3.4960676366496264
15등 영화 장르 : Action, 3.484013230429989
16등 영화 장르 : Children's, 3.4269713261648747
17등 영화 장르 : Comedy, 3.424020823004462
18등 영화 장르 : Horror, 3.263993316624896
19등 영화 장르 : Fantasy, 3.201101928374656

남자 사용자가 평가한 영화 장르의 평균 평점을 내림차순으로

1등 영화 장르 : Film-Noir, 3.973293768545994
2등 영화 장르 : War, 3.826328200860036
3등 영화 장르 : Drama, 3.6969571087340327
4등 영화 장르 : Documentary, 3.691768826619965
5등 영화 장르 : Mystery, 3.6642075807682524
6등 영화 장르 : 

### B-3. Measure the average ratings by genre for all occupations and sort them in descending order respectively

In [84]:
#직업별로 사용자가 평가한 영화 장르의 평점의 평균을 구한 후 내림차순하여라(큰거 -> 작은거)

#u_item과 u_data를 inner joing (on movie id)한다.
join_data = pd.merge(u_data, u_item)

#occupation_user과 join_dta를 inner joing한다.
occupation_user = u_user.loc[:,["user id","occupation"]]
join_data = pd.merge(join_data, occupation_user)

for idx in range(len(occupation)):
    occupation_data = join_data.loc[join_data['occupation']==occupation['job'][idx]] #occupation별로 데이터 뽑기
     #genre_data에 장르와 평점의 평균을 저장한다.
    genre_data = []
    for i in range(len(genre)-1):
        temp = []
        #현재 다루는 장르
        current_genre = genre['genre_name'][i]
        #현재 장르를 평가한 데이터를 뽑아낸다.
        specific_genre = occupation_data.loc[occupation_data[current_genre]==1,[current_genre,'rating']]
        genre_mean = specific_genre['rating'].mean()
        temp.append(current_genre)
        temp.append(genre_mean)
        genre_data.append(temp)

    genre_data = pd.DataFrame(genre_data,columns=['genre_name','mean'])
    #평균의 평점을 내림차순 정렬한다.
    genre_data = genre_data.sort_values(by='mean', ascending=False).reset_index(drop=True)
    
    print("\n{0}. {1}이 평가한 영화 장르의 평점 평균을 내림차순으로 정렬\n".format(idx+1,occupation['job'][idx]))
    
    for i in range(len(genre_data)):
        print("{0}등 영화 장르 : {1}, {2}".format(i+1, genre_data['genre_name'][i],genre_data['mean'][i]))
        


1. administrator이 평가한 영화 장르의 평점 평균을 내림차순으로 정렬

1등 영화 장르 : Film-Noir, 3.9727891156462585
2등 영화 장르 : War, 3.9249363867684477
3등 영화 장르 : Documentary, 3.875
4등 영화 장르 : Drama, 3.7960632462084543
5등 영화 장르 : Mystery, 3.7636363636363637
6등 영화 장르 : Romance, 3.741390513320338
7등 영화 장르 : Animation, 3.7389162561576357
8등 영화 장르 : Western, 3.7023809523809526
9등 영화 장르 : Crime, 3.6938053097345134
10등 영화 장르 : Sci-Fi, 3.650743099787686
11등 영화 장르 : Thriller, 3.6044585987261146
12등 영화 장르 : Musical, 3.596045197740113
13등 영화 장르 : Action, 3.538751345532831
14등 영화 장르 : Adventure, 3.5215686274509803
15등 영화 장르 : Comedy, 3.4947798456650023
16등 영화 장르 : Fantasy, 3.4680851063829787
17등 영화 장르 : Children's, 3.448051948051948
18등 영화 장르 : Horror, 3.39386189258312
19등 영화 장르 : unknown, nan

2. artist이 평가한 영화 장르의 평점 평균을 내림차순으로 정렬

1등 영화 장르 : Film-Noir, 4.234375
2등 영화 장르 : Documentary, 4.147058823529412
3등 영화 장르 : War, 3.902222222222222
4등 영화 장르 : Western, 3.8620689655172415
5등 영화 장르 : Sci-Fi, 3.8181818181818183
6등 영화 장르 


13. none이 평가한 영화 장르의 평점 평균을 내림차순으로 정렬

1등 영화 장르 : unknown, 5.0
2등 영화 장르 : War, 4.072727272727272
3등 영화 장르 : Documentary, 4.0
4등 영화 장르 : Animation, 3.9767441860465116
5등 영화 장르 : Drama, 3.9328358208955225
6등 영화 장르 : Romance, 3.92
7등 영화 장르 : Musical, 3.86046511627907
8등 영화 장르 : Horror, 3.857142857142857
9등 영화 장르 : Thriller, 3.83587786259542
10등 영화 장르 : Sci-Fi, 3.806896551724138
11등 영화 장르 : Children's, 3.7252747252747254
12등 영화 장르 : Action, 3.7242524916943522
13등 영화 장르 : Adventure, 3.70625
14등 영화 장르 : Comedy, 3.6784313725490194
15등 영화 장르 : Crime, 3.5945945945945947
16등 영화 장르 : Fantasy, 3.5833333333333335
17등 영화 장르 : Mystery, 3.5454545454545454
18등 영화 장르 : Film-Noir, 3.5
19등 영화 장르 : Western, 3.5

14. other이 평가한 영화 장르의 평점 평균을 내림차순으로 정렬

1등 영화 장르 : unknown, 4.0
2등 영화 장르 : Film-Noir, 3.8424657534246576
3등 영화 장르 : War, 3.842
4등 영화 장르 : Documentary, 3.7846153846153845
5등 영화 장르 : Drama, 3.7192941722776873
6등 영화 장르 : Animation, 3.672209026128266
7등 영화 장르 : Western, 3.627906976744186
8등 영화 장르 : Mu